# The Alhazen Agent

> Using an agent to chat with a user in order to distinguish between tasks. 

In [1]:
#| default_exp agent.gradio_chat

In [1]:
#| export

import os
import pandas as pd

import alhazen.utils.jats_text_extractor as te 
import re



from langchain.llms import LlamaCpp
from langchain.llms import OpenAI

from langchain.agents import initialize_agent , Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

import requests

import gradio as gr
from databricks import sql

from importlib_resources import files

import fire

In [3]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import BaseMessage
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage
from langchain.schema import AgentAction, AgentFinish
from langchain.schema.output import LLMResult
from langchain.utils import print_text

from typing import (
    Any,
    Dict,
    List,
    Union,
    Optional
)

class AlhazenCallbackHandler(BaseCallbackHandler):
    def __init__(self, color: Optional[str] = None) -> None:
        """Initialize callback handler."""
        self.color = color

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> None:
        """Print out the prompts."""
        for p in prompts:
            print(f"> Prompt: {p}")

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        """Do nothing."""
        pass

    def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
        """Do nothing."""
        pass

    def on_llm_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> None:
        """Do nothing."""
        pass

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> None:
        """Print out that we are entering a chain."""
        class_name = serialized.get("name", serialized.get("id", ["<unknown>"])[-1])
        print(f"\n\n\033[1m> Entering new {class_name} chain...\033[0m")

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs: Any) -> None:
        """Print out that we finished a chain."""
        print("\n\033[1m> Finished chain.\033[0m")

    def on_chain_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> None:
        """Do nothing."""
        pass

    def on_tool_start(
        self,
        serialized: Dict[str, Any],
        input_str: str,
        **kwargs: Any,
    ) -> None:
        """Do nothing."""
        pass

    def on_agent_action(
        self, action: AgentAction, color: Optional[str] = None, **kwargs: Any
    ) -> Any:
        """Run on agent action."""
        print_text(action.log, color=color or self.color)

    def on_tool_end(
        self,
        output: str,
        color: Optional[str] = None,
        observation_prefix: Optional[str] = None,
        llm_prefix: Optional[str] = None,
        **kwargs: Any,
    ) -> None:
        """If not the final action, print out observation."""
        if observation_prefix is not None:
            print_text(f"\n{observation_prefix}")
        print_text(output, color=color or self.color)
        if llm_prefix is not None:
            print_text(f"\n{llm_prefix}")

    def on_tool_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> None:
        """Do nothing."""
        pass

    def on_text(
        self,
        text: str,
        color: Optional[str] = None,
        end: str = "",
        **kwargs: Any,
    ) -> None:
        """Run when agent ends."""
        print_text(text, color=color or self.color, end=end)

    def on_agent_finish(
        self, finish: AgentFinish, color: Optional[str] = None, **kwargs: Any
    ) -> None:
        """Run on agent end."""
        print_text(finish.log, color=color or self.color, end="\n")


In [4]:
#| export 

from langchain.agents import tool,  initialize_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import MessagesPlaceholder
from langchain.schema.messages import HumanMessage, AIMessage

from langchain.agents import AgentOutputParser
from langchain.agents.conversational_chat.prompt import FORMAT_INSTRUCTIONS
from langchain.output_parsers.json import parse_json_markdown
from langchain.schema import AgentAction, AgentFinish
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

import time 
import pprint

GGUF_LOOKUP_URL = {
    "llama-2-70b-chat.Q5_K_M.gguf": "https://huggingface.co/TheBloke/Llama-2-70B-chat-GGUF/resolve/main/llama-2-70b-chat.Q4_K_M.gguf",
    "llama-2-13b-chat.Q5_K_M.gguf": "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf",
    "llama-2-7b-chat.Q5_K_M.gguf": "https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q5_K_M.gguf"
}

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"

sys_msg = B_SYS + """Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to respond to the User and use tools using JSON strings that contain "action" and "action_input" parameters.

All of Assistant's communication is performed using this JSON format.

Assistant can also use tools by responding to the user with tool use instructions in the same "action" and "action_input" JSON format. Tools available to Assistant are:

- "Calculator": Useful for when you need to answer questions about math.
  - To use the calculator tool, Assistant should write like so:
    ```json
    {{"action": "Calculator",
      "action_input": "sqrt(4)"}}
    ```
    
Here are some previous conversations between the Assistant and User:

User: Hey how are you today?
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "I'm good thanks, how are you?"}}
```
User: I'm great, what is the square root of 4?
Assistant: ```json
{{"action": "Calculator",
 "action_input": "sqrt(4)"}}
```
User: 2.0
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "It looks like the answer is 2!"}}
```
User: Thanks could you tell me what 4 to the power of 2 is?
Assistant: ```json
{{"action": "Calculator",
 "action_input": "4**2"}}
```
User: 16.0
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "It looks like the answer is 16!"}}
```

Here is the latest conversation between Assistant and User.""" + E_SYS

from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import load_tools
from langchain.schema import LLMResult

class AlhazenAgent:

    def __init__(self, temp_dir='/tmp/alhazen',
                gguf_file_name='llama-2-13b-chat.Q5_K_M.gguf',
                temperature=0.1,
                context_window=4096):
        
        if temp_dir[-1:] != '/':
            temp_dir += '/'
        self.temp_dir = temp_dir

        # download relevant file to local disk if not already there
        if os.path.exists(temp_dir+gguf_file_name) is False:
            print('Downloading GGUF file: ' + gguf_file_name)
            os.makedirs(temp_dir, exist_ok=True)

            # download file from HuggingFace but show progress bar  
            r = requests.get(GGUF_LOOKUP_URL[gguf_file_name], stream=True)
            with open(temp_dir+gguf_file_name, 'wb') as f:
                total_length = int(r.headers.get('content-length'))
                dl = 0
                for chunk in r.iter_content(chunk_size=1024*1024):
                    dl += len(chunk)
                    f.write(chunk)
                    done = int(50 * dl / total_length)
                    print('\r[{}{}]'.format('=' * done, ' ' * (50-done)), end='')
                print('\n')
            print('Download complete.')
                        
        n_gpu_layers = 1  # Change this value based on your model and your GPU VRAM pool.
        n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

        self.llm = LlamaCpp(model_path=temp_dir+gguf_file_name,
                            n_ctx=context_window,
                            n_gpu_layers=n_gpu_layers,
                            temperature=temperature,
                            n_batch=n_batch,
                            verbose=True)
        
        self.memory = ConversationBufferWindowMemory(
            memory_key="chat_history", 
            k=5, 
            return_messages=True, 
            output_key="output")
        
        self.tools = load_tools(["llm-math"], llm=self.llm)

        class OutputParser(AgentOutputParser):                    
            def get_format_instructions(self) -> str:
                return FORMAT_INSTRUCTIONS

            def parse(self, text: str) -> AgentAction | AgentFinish:
                try:
                    # this will work IF the text is a valid JSON with action and action_input
                    response = parse_json_markdown(text)
                    action, action_input = response["action"], response["action_input"]
                    if action == "Final Answer":
                        # this means the agent is finished so we call AgentFinish
                        return AgentFinish({"output": action_input}, text)
                    else:
                        # otherwise the agent wants to use an action, so we call AgentAction
                        return AgentAction(action, action_input, text)
                except Exception:
                    # sometimes the agent will return a string that is not a valid JSON
                    # often this happens when the agent is finished
                    # so we just return the text as the output
                    return AgentFinish({"output": text}, text)

            @property
            def _type(self) -> str:
                return "conversational_chat"

        # initialize output parser for agent
        parser = OutputParser()

        # initialize agent
        callback_manager = CallbackManager([AlhazenCallbackHandler()])
        self.agent = initialize_agent(
            agent="chat-conversational-react-description",
            tools=self.tools,
            llm=self.llm,
            verbose=True,
            early_stopping_method="generate",
            memory=self.memory,
            agent_kwargs={"output_parser": parser},
            callback_manager=callback_manager
        )
        new_prompt = self.agent.agent.create_prompt(
            system_message=sys_msg,
            tools=self.tools
        )
        self.agent.agent.llm_chain.prompt = new_prompt

        instruction = B_INST + " Respond to the following in JSON with 'action' and 'action_input' values " + E_INST
        human_msg = instruction + "\nUser: {input}"
        self.agent.agent.llm_chain.prompt.messages[2].prompt.template = human_msg

    def run_gradio(self):

        def add_text(history, text):
            #print('add_text: history: %s, text: %s'%(history, text))
            history = history + [(text, None)]
            return history, gr.Textbox(value="", interactive=False)

        def add_file(history, file):
            #print('add_history: history: %s, file: %s'%(history, file))
            history = history + [((file.name,), None)]
            return history

        def bot(history):
            #print('bot: history: %s'%(history))
            # prompt to send to the agent is the last message from the user
            prompt = history[-1][0]
            agent_output = self.agent(prompt)
            response = agent_output.get('output','ERROR')
            history[-1][1] = response
            return history

        with gr.Blocks() as demo:
            chatbot = gr.Chatbot(
                [],
                elem_id="chatbot",
                bubble_full_width=False,
                #avatar_images=(None, files(alhazen_resources).joinpath('alhazen.png'))
            )
            with gr.Row():
                txt = gr.Textbox(
                    scale=4,
                    show_label=False,
                    placeholder="Enter text and press enter, or upload files",
                    container=False,
                )
                btn = gr.UploadButton("📁", file_types=["image", "video", "audio"])

            txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(bot, chatbot, chatbot)
            txt_msg.then(lambda: gr.Textbox(interactive=True), None, [txt], queue=False)
            
            file_msg = btn.upload(add_file, [chatbot, btn], [chatbot], queue=False).then(bot, chatbot, chatbot)

        demo.queue()
        demo.launch()

alhazen = AlhazenAgent(temp_dir='/tmp/alhazen',
                gguf_file_name='llama-2-70b-chat.Q5_K_M.gguf')


llama_model_loader: loaded meta data with 19 key-value pairs and 723 tensors from /tmp/alhazen/llama-2-70b-chat.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  8192, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 28672,  8192,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  8192,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  8192,  8192, 

In [6]:
alhazen.run_gradio()

Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: <>
Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to respond to the User and use tools using JSON strings that contain "action" and "action_input" parameters.

All of Assistant's communication is performed using this JSON format.

Assistant can also use tools by responding to the user with tool use instructions in the same "action" and "action_input" JSON format. Tools available to Assistant are:

- "Calculator": Useful for when you need to answer questions about math.
  - To use the calculator tool, Assistant should write like so:
    ```json
    {"action": "Calculator",
      "action_input": "sqrt(4)"}
    ```
    
Here are some previous conversations between the Assistant and User:

User: Hey how are you today?
Assistant: ```json
{"action": "Final Answer",
 "action_input": "I'm good t


llama_print_timings:        load time = 10856.01 ms
llama_print_timings:      sample time =    27.69 ms /    39 runs   (    0.71 ms per token,  1408.25 tokens per second)
llama_print_timings: prompt eval time = 10855.77 ms /   472 tokens (   23.00 ms per token,    43.48 tokens per second)
llama_print_timings:        eval time =  6210.17 ms /    38 runs   (  163.43 ms per token,     6.12 tokens per second)
llama_print_timings:       total time = 17142.58 ms
Llama.generate: prefix-match hit



> Finished chain.
?
Assistant: ```json
{"action": "Calculator",
 "action_input": "10000+9813740"}
```?
Assistant: ```json
{"action": "Calculator",
 "action_input": "10000+9813740"}
```


llama_print_timings:        load time = 10856.01 ms
llama_print_timings:      sample time =    32.59 ms /    46 runs   (    0.71 ms per token,  1411.65 tokens per second)
llama_print_timings: prompt eval time =  5257.58 ms /   283 tokens (   18.58 ms per token,    53.83 tokens per second)
llama_print_timings:        eval time =  7718.45 ms /    45 runs   (  171.52 ms per token,     5.83 tokens per second)
llama_print_timings:       total time = 13067.62 ms
Llama.generate: prefix-match hit



Observation: Answer: 9823740
Thought:

> Entering new LLMChain chain...
Prompt after formatting:
System: <>
Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to respond to the User and use tools using JSON strings that contain "action" and "action_input" parameters.

All of Assistant's communication is performed using this JSON format.

Assistant can also use tools by responding to the user with tool use instructions in the same "action" and "action_input" JSON format. Tools available to Assistant are:

- "Calculator": Useful for when you need to answer questions about math.
  - To use the calculator tool, Assistant should write like so:
    ```json
    {"action": "Calculator",
      "action_input": "sqrt(4)"}
    ```
    
Here are some previous conversations between the Assistant and User:

User: Hey how are you today?
Assistant: ```json
{"action": "Final Answer",
 "action_input": "I'm good thanks, how are you?"}
```
User: I'm great,


llama_print_timings:        load time = 10856.01 ms
llama_print_timings:      sample time =    26.16 ms /    37 runs   (    0.71 ms per token,  1414.10 tokens per second)
llama_print_timings: prompt eval time = 12334.48 ms /   620 tokens (   19.89 ms per token,    50.27 tokens per second)
llama_print_timings:        eval time =  6394.79 ms /    36 runs   (  177.63 ms per token,     5.63 tokens per second)
llama_print_timings:       total time = 18804.00 ms



> Finished chain.


Assistant: ```json
{"action": "Final Answer",
 "action_input": "The answer is 9823740"}
```


Assistant: ```json
{"action": "Final Answer",
 "action_input": "The answer is 9823740"}
```

> Finished chain.

> Finished chain.


In [25]:
import pprint

pprint.pprint(alhazen.agent.agent.llm_chain.prompt.__dict__, depth=2, width=60)

{'input_variables': ['input',
                     'chat_history',
                     'agent_scratchpad'],
 'messages': [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='<>\nAssistant is a expert JSON builder designed to assist with a wide range of tasks.\n\nAssistant is able to respond to the User and use tools using JSON strings that contain "action" and "action_input" parameters.\n\nAll of Assistant\'s communication is performed using this JSON format.\n\nAssistant can also use tools by responding to the user with tool use instructions in the same "action" and "action_input" JSON format. Tools available to Assistant are:\n\n- "Calculator": Useful for when you need to answer questions about math.\n  - To use the calculator tool, Assistant should write like so:\n    ```json\n    {{"action": "Calculator",\n      "action_input": "sqrt(4)"}}\n    ```\n\nHere are some previous conversations between the Assistant and User:\n\nUser: Hey how are you today?\nA

In [27]:
alhazen.agent('Can you compute the square root of 81?')



> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit



Assistant: ```json
{"action": "Calculator",
 "action_input": "sqrt(81)"}
```
Assistant: ```json
{"action": "Calculator",
 "action_input": "sqrt(81)"}
```


llama_print_timings:        load time = 10620.77 ms
llama_print_timings:      sample time =    21.77 ms /    30 runs   (    0.73 ms per token,  1378.11 tokens per second)
llama_print_timings: prompt eval time =  3024.41 ms /    63 tokens (   48.01 ms per token,    20.83 tokens per second)
llama_print_timings:        eval time =  4722.84 ms /    29 runs   (  162.86 ms per token,     6.14 tokens per second)
llama_print_timings:       total time =  7817.16 ms
Llama.generate: prefix-match hit


```text
sqrt(81)
```
...numexpr.evaluate("sqrt(81)")...



llama_print_timings:        load time = 10620.77 ms
llama_print_timings:      sample time =    20.00 ms /    28 runs   (    0.71 ms per token,  1400.21 tokens per second)
llama_print_timings: prompt eval time =  5160.08 ms /   275 tokens (   18.76 ms per token,    53.29 tokens per second)
llama_print_timings:        eval time =  4377.82 ms /    27 runs   (  162.14 ms per token,     6.17 tokens per second)
llama_print_timings:       total time =  9602.01 ms
Llama.generate: prefix-match hit



Observation: Answer: 9.0
Thought:

AI: 
Assistant: ```json
{"action": "Final Answer",
 "action_input": "It looks like the answer is 9!"}
```

AI: 
Assistant: ```json
{"action": "Final Answer",
 "action_input": "It looks like the answer is 9!"}
```

> Finished chain.



llama_print_timings:        load time = 10620.77 ms
llama_print_timings:      sample time =    27.79 ms /    39 runs   (    0.71 ms per token,  1403.28 tokens per second)
llama_print_timings: prompt eval time = 12615.09 ms /   625 tokens (   20.18 ms per token,    49.54 tokens per second)
llama_print_timings:        eval time =  6482.36 ms /    38 runs   (  170.59 ms per token,     5.86 tokens per second)
llama_print_timings:       total time = 19188.24 ms


{'input': 'Can you compute the square root of 81?', 'chat_history': [HumanMessage(content='Can you compute 1+7?'), AIMessage(content='It looks like the answer is 8!')], 'output': 'It looks like the answer is 9!'}

In [28]:
#| export

def ask_alhazen(temp_dir='/tmp/alhazen',
                gguf_file_name='llama-2-70b-chat.Q5_K_M.gguf',
                temperature=0.1,
                context_window=4096):
    
    chatbot = AlhazenAgent(temp_dir=temp_dir,
                gguf_file_name=gguf_file_name,
                temperature=temperature,
                context_window=context_window)

    chatbot.run_gradio()

#chatbot()


In [29]:
ask_alhazen()

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/alhazen/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  5120,  5120, 

NameError: name '__file__' is not defined

## How to run this chatbot from the command line 

```
python -m fire alhazen/agent/gradio_chat.py chatbot
```

In [25]:

import sys
print(sys.path)
from datasets import load_dataset

['/Users/gburns/Documents/Coding/ChatGPT_etc/alzhazen/nbs', '/Users/gburns/miniconda3/envs/alhazen/lib/python311.zip', '/Users/gburns/miniconda3/envs/alhazen/lib/python3.11', '/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/lib-dynload', '', '/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/site-packages', '/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/site-packages/mlc_llm-0.1.dev333+g2440594-py3.11.egg', '/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/site-packages/safetensors-0.3.2rc1-py3.11-macosx-11.1-arm64.egg', '/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/site-packages/regex-2023.6.3-py3.11-macosx-11.1-arm64.egg', '/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/site-packages/networkx-3.1-py3.11.egg', '/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/site-packages/sympy-1.12-py3.11.egg', '/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/site-packages/mpmath-1.3.0-py3.11.egg', '/Users/gburns/Documents/Coding/ChatGPT_etc/alzhazen', '/var

ImportError: cannot import name 'load_dataset' from 'datasets' (unknown location)

In [12]:
from langchain.base_language import BaseLanguageModel
from langchain.tools.base import BaseTool
from typing import Any, List, Optional

from langchain import LLMChain, PromptTemplate
from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import Callbacks
import copy
import uuid
from typing import Dict, List

import numpy as np
from langchain.tools.base import BaseTool

import json
import re
from abc import abstractmethod
from typing import Any, Dict, List, Optional, Union

from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import Callbacks
from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.tools.base import BaseTool
from pydantic import BaseModel

class HuggingGPT:
    def __init__(self, llm: BaseLanguageModel, tools: List[BaseTool]):
        self.llm = llm
        self.tools = tools
        self.chat_planner = load_chat_planner(llm)
        self.response_generator = load_response_generator(llm)
        self.task_executor: TaskExecutor

    def run(self, input: str) -> str:
        plan = self.chat_planner.plan(inputs={"input": input, "hf_tools": self.tools})
        self.task_executor = TaskExecutor(plan)
        self.task_executor.run()
        response = self.response_generator.generate(
            {"task_execution": self.task_executor}
        )
        return response

class ResponseGenerationChain(LLMChain):
    """Chain to execute tasks."""

    @classmethod
    def from_llm(cls, llm: BaseLanguageModel, verbose: bool = True) -> LLMChain:
        execution_template = (
            "The AI assistant has parsed the user input into several tasks"
            "and executed them. The results are as follows:\n"
            "{task_execution}"
            "\nPlease summarize the results and generate a response."
        )
        prompt = PromptTemplate(
            template=execution_template,
            input_variables=["task_execution"],
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

class ResponseGenerator:
    def __init__(self, llm_chain: LLMChain, stop: Optional[List] = None):
        self.llm_chain = llm_chain
        self.stop = stop

    def generate(self, inputs: dict, callbacks: Callbacks = None, **kwargs: Any) -> str:
        """Given input, decided what to do."""
        llm_response = self.llm_chain.run(**inputs, stop=self.stop, callbacks=callbacks)
        return llm_response

def load_response_generator(llm: BaseLanguageModel) -> ResponseGenerator:
    llm_chain = ResponseGenerationChain.from_llm(llm)
    return ResponseGenerator(
        llm_chain=llm_chain,
    )

class TaskPlanningChain(LLMChain):
    """Chain to execute tasks."""

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        verbose: bool = True,
    ) -> LLMChain:
        """Get the response parser."""
        system_template = """ 
                Your task is to break down a users request into a list of several tasks formatted as JSON: 
                [{{"task": task, "id": task_id, "dep": dependency_task_id, 
                    "args": {{"input name": text may contain <resource-dep_id>}}}}]. 
                The special tag "dep_id" refer to the one generated text/image/audio in the dependency task 
                (Please consider whether the dependency task generates resources of this type.) 
                and "dep_id" must be in "dep" list. 
                The "dep" field denotes the ids of the previous prerequisite tasks which 
                generate a new resource that the current task relies on. 
                The task MUST be selected from the following tools 
                (along with tool description, input name and output type): 
                    {tools}. 
                There may be multiple tasks of the same type. 
                Take a deep breath and think carefully about the user's request.
                Then think step by step about all the tasks needed to resolve the user's request. 
                Parse out as few tasks as possible while ensuring that the user request can be resolved. 
                Pay attention to the dependencies and order among tasks. 
                Remember, you must ONLY return a list of tasks formatted as JSON.
                If the user input can't be parsed, you need to reply empty JSON []."""  # noqa: E501
        human_template = """Now I input: {input}."""
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )
        return cls(prompt=prompt, llm=llm, verbose=verbose)

class Step:
    def __init__(
        self, task: str, id: int, dep: List[int], args: Dict[str, str], tool: BaseTool
    ):
        self.task = task
        self.id = id
        self.dep = dep
        self.args = args
        self.tool = tool


class Plan:
    def __init__(self, steps: List[Step]):
        self.steps = steps

    def __str__(self) -> str:
        return str([str(step) for step in self.steps])

    def __repr__(self) -> str:
        return str(self)


class BasePlanner(BaseModel):
    @abstractmethod
    def plan(self, inputs: dict, callbacks: Callbacks = None, **kwargs: Any) -> Plan:
        """Given input, decide what to do."""

    @abstractmethod
    async def aplan(
        self, inputs: dict, callbacks: Callbacks = None, **kwargs: Any
    ) -> Plan:
        """Given input, decide what to do."""


class PlanningOutputParser(BaseModel):
    def parse(self, text: str, hf_tools: List[BaseTool]) -> Plan:
        steps = []
        s = text.find("[")
        e = text.rfind("]") + 1
        for v in json.loads(text[s:e]):
            choose_tool = None
            for tool in hf_tools:
                if tool.name == v["task"]:
                    choose_tool = tool
                    break
            if choose_tool:
                steps.append(Step(v["task"], v["id"], v["dep"], v["args"], tool))
        return Plan(steps=steps)


class TaskPlanner(BasePlanner):
    llm_chain: LLMChain
    output_parser: PlanningOutputParser
    stop: Optional[List] = None

    def plan(self, inputs: dict, callbacks: Callbacks = None, **kwargs: Any) -> Plan:
        """Given input, decided what to do."""
        inputs["tools"] = [
            f"{tool.name}: {tool.description}" for tool in inputs["hf_tools"]
        ]
        llm_response = self.llm_chain.run(**inputs, stop=self.stop, callbacks=callbacks)
        return self.output_parser.parse(llm_response, inputs["hf_tools"])

    async def aplan(
        self, inputs: dict, callbacks: Callbacks = None, **kwargs: Any
    ) -> Plan:
        """Given input, decided what to do."""
        inputs["hf_tools"] = [
            f"{tool.name}: {tool.description}" for tool in inputs["hf_tools"]
        ]
        llm_response = await self.llm_chain.arun(
            **inputs, stop=self.stop, callbacks=callbacks
        )
        return self.output_parser.parse(llm_response, inputs["hf_tools"])


def load_chat_planner(llm: BaseLanguageModel) -> TaskPlanner:
    llm_chain = TaskPlanningChain.from_llm(llm)
    return TaskPlanner(llm_chain=llm_chain, output_parser=PlanningOutputParser())

class Task:
    def __init__(self, task: str, id: int, dep: List[int], args: Dict, tool: BaseTool):
        self.task = task
        self.id = id
        self.dep = dep
        self.args = args
        self.tool = tool
        self.status = "pending"
        self.message = ""
        self.result = ""

    def __str__(self) -> str:
        return f"{self.task}({self.args})"

    def save_product(self) -> None:
        import cv2

        if self.task == "video_generator":
            # ndarray to video
            product = np.array(self.product)
            nframe, height, width, _ = product.shape
            video_filename = uuid.uuid4().hex[:6] + ".mp4"
            fps = 30  # Frames per second
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # type: ignore
            video_out = cv2.VideoWriter(video_filename, fourcc, fps, (width, height))
            for frame in self.product:
                video_out.write(frame)
            video_out.release()
            self.result = video_filename
        elif self.task == "image_generator":
            # PIL.Image to image
            filename = uuid.uuid4().hex[:6] + ".png"
            self.product.save(filename)  # type: ignore
            self.result = filename
        

    def completed(self) -> bool:
        return self.status == "completed"

    def failed(self) -> bool:
        return self.status == "failed"

    def pending(self) -> bool:
        return self.status == "pending"

    def run(self) -> str:
        from diffusers.utils import load_image

        try:
            new_args = copy.deepcopy(self.args)
            for k, v in new_args.items():
                if k == "image":
                    new_args["image"] = load_image(v)
            if self.task in ["video_generator", "image_generator", "text_reader"]:
                self.product = self.tool(**new_args)
            else:
                self.result = self.tool(**new_args)
        except Exception as e:
            self.status = "failed"
            self.message = str(e)
        self.status = "completed"
        self.save_product()

        return self.result

class TaskExecutor:
    """Load tools to execute tasks."""

    def __init__(self, plan: Plan):
        self.plan = plan
        self.tasks = []
        self.id_task_map = {}
        self.status = "pending"
        for step in self.plan.steps:
            task = Task(step.task, step.id, step.dep, step.args, step.tool)
            self.tasks.append(task)
            self.id_task_map[step.id] = task

    def completed(self) -> bool:
        return all(task.completed() for task in self.tasks)

    def failed(self) -> bool:
        return any(task.failed() for task in self.tasks)

    def pending(self) -> bool:
        return any(task.pending() for task in self.tasks)

    def check_dependency(self, task: Task) -> bool:
        for dep_id in task.dep:
            if dep_id == -1:
                continue
            dep_task = self.id_task_map[dep_id]
            if dep_task.failed() or dep_task.pending():
                return False
        return True

    def update_args(self, task: Task) -> None:
        for dep_id in task.dep:
            if dep_id == -1:
                continue
            dep_task = self.id_task_map[dep_id]
            for k, v in task.args.items():
                if f"<resource-{dep_id}>" in v:
                    task.args[k].replace(f"<resource-{dep_id}>", dep_task.result)

    def run(self) -> str:
        for task in self.tasks:
            print(f"running {task}")
            if task.pending() and self.check_dependency(task):
                self.update_args(task)
                task.run()
        if self.completed():
            self.status = "completed"
        elif self.failed():
            self.status = "failed"
        else:
            self.status = "pending"
        return self.status

    def __str__(self) -> str:
        result = ""
        for task in self.tasks:
            result += f"{task}\n"
            result += f"status: {task.status}\n"
            if task.failed():
                result += f"message: {task.message}\n"
            if task.completed():
                result += f"result: {task.result}\n"
        return result

    def __repr__(self) -> str:
        return self.__str__()

    def describe(self) -> str:
        return self.__str__()


In [13]:
from transformers import load_tool

hf_tools = [
    load_tool(tool_name)
    for tool_name in [
        "document-question-answering",
        "image-captioning",
        "image-question-answering",
        "image-segmentation",
        "summarization",
        "text-classification",
        "text-question-answering",
        "translation",
        "huggingface-tools/text-to-image",
        "huggingface-tools/text-to-video",
        "huggingface-tools/text-download",
        "huggingface-tools/image-transformation",
    ]
]

n_gpu_layers = 1  # Change this value based on your model and your GPU VRAM pool.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.

llm = LlamaCpp(model_path='/tmp/alhazen/llama-2-70b-chat.Q5_K_M.gguf',
                n_ctx=4096,
                n_gpu_layers=1,
                temperature=0.1,
                n_batch=512,
                verbose=True)

agent = HuggingGPT(llm, hf_tools)

llama_model_loader: loaded meta data with 19 key-value pairs and 723 tensors from /tmp/alhazen/llama-2-70b-chat.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  8192, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 28672,  8192,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  8192,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  8192,  8192, 

In [ ]:
from typing import Sequence
from langchain.schema import BaseMessage, HumanMessage, AIMessage, SystemMessage, FunctionMessage, ChatMessage

def get_buffer_string_for_llama(
    messages: Sequence[BaseMessage], human_prefix: str = "Human", ai_prefix: str = "AI"
) -> str:
    """Convert sequence of Messages to strings and concatenate them into one string.

    Args:
        messages: Messages to be converted to strings.
        human_prefix: The prefix to prepend to contents of HumanMessages.
        ai_prefix: THe prefix to prepend to contents of AIMessages.

    Returns:
        A single string concatenation of all input messages, 
        formatted for Llama Chat Models with <<SYS>>, <</SYS>>, [INST], and [/INST] tags.

    Example:
        .. code-block:: python

            from langchain.schema import AIMessage, HumanMessage

            messages = [
                HumanMessage(content="Hi, how are you?"),
                AIMessage(content="Good, how are you?"),
            ]
            get_buffer_string(messages)
            "<s><<SYS>>You are a helpful bot<</SYS>>
            [INST] Hi, how are you?[/INST]
            Good, how are you?"</s>
            <s>[INST]I'm fine, can you tell me who wrote 'Arabian Nights?'[/INST]
            Antoine Galland has been for three centuries, credited as being the author of the Arabian Nights.</s>
            ...
    """
    string_messages = []
    first_block = True 
    for m in messages:
        if isinstance(m, HumanMessage):
            if first_block:
                b = '[INST]' # assume the first message in the chain includes a system prompt and the remainders do not. 
                first_block = False 
            else:
                b = '<s>[INST]'
            e = '[/INST]'
        elif isinstance(m, AIMessage):
            b = ''
            e = '</s>' # assume the AI message is the last message of each block
        elif isinstance(m, SystemMessage):
            b = "<s><<SYS>>" # assume the systems message is the first message
            e = "<</SYS>>"
        elif isinstance(m, FunctionMessage):
            b = ''
            e = ''
        elif isinstance(m, ChatMessage):
            b = ''
            e = ''
        else:
            raise ValueError(f"Got unsupported message type: {m}")
        message = f"{b}{m.content}{e}"
        if isinstance(m, AIMessage) and "function_call" in m.additional_kwargs:
            message += f"{m.additional_kwargs['function_call']}"
        string_messages.append(message)

    return "\n".join(string_messages)

def ChatPromptValue_to_string(self) -> str:
    """Return prompt as string."""
    return get_buffer_string_for_llama(self.messages)

from langchain.prompts.chat import ChatPromptValue
ChatPromptValue.to_string = ChatPromptValue_to_string


In [16]:
agent.run("please show me a video and an image of 'a boy is running'")



> Entering new TaskPlanningChain chain...
Prompt after formatting:
<<SYS>> 
                Your task is to break down a users request into a list of several tasks formatted as JSON: 
                [{"task": task, "id": task_id, "dep": dependency_task_id, 
                    "args": {"input name": text may contain <resource-dep_id>}}]. 
                The special tag "dep_id" refer to the one generated text/image/audio in the dependency task 
                (Please consider whether the dependency task generates resources of this type.) 
                and "dep_id" must be in "dep" list. 
                The "dep" field denotes the ids of the previous prerequisite tasks which 
                generate a new resource that the current task relies on. 
                The task MUST be selected from the following tools 
                (along with tool description, input name and output type): 
                    ['document_qa: This is a tool that answers a question about an docume

Llama.generate: prefix-match hit



> Finished chain.



llama_print_timings:        load time = 10416.40 ms
llama_print_timings:      sample time =   188.94 ms /   256 runs   (    0.74 ms per token,  1354.96 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 47248.86 ms /   256 runs   (  184.57 ms per token,     5.42 tokens per second)
llama_print_timings:       total time = 47856.42 ms


In [1]:
import gradio as gr
from gradio_pdf import PDF
from pdf2image import convert_from_path
from transformers import pipeline
from pathlib import Path

dir_ = Path(__file__).parent

p = pipeline(
    "document-question-answering",
    model="impira/layoutlm-document-qa",
)

def qa(question: str, doc: str) -> str:
    img = convert_from_path(doc)[0]
    output = p(img, question)
    return sorted(output, key=lambda x: x["score"], reverse=True)[0]['answer']


demo = gr.Interface(
    qa,
    [gr.Textbox(label="Question"), PDF(label="Document")],
    gr.Textbox(),
    examples=[["What is the total gross worth?", str(dir_ / "invoice_2.pdf")],
              ["Whos is being invoiced?", str(dir_ / "sample_invoice.pdf")]]
)

demo.launch()

ModuleNotFoundError: No module named 'gradio_pdf'